In [1]:
!pip install pyglet

  Using cached pyglet-1.5.26-py3-none-any.whl (1.1 MB)


In [2]:
!pip install gTTS

  Using cached gTTS-2.2.4-py3-none-any.whl (26 kB)


In [3]:
!pip install mediapipe

  Using cached mediapipe-0.8.10.1-cp39-cp39-win_amd64.whl (48.7 MB)
  Using cached opencv_contrib_python-4.6.0.66-cp36-abi3-win_amd64.whl (42.5 MB)


In [11]:
import cv2
import mediapipe as mp
import numpy as np
import os
from gtts import gTTS
import pyglet
from time import sleep
import datetime
from datetime import timedelta

language = 'en'
mp_drawing = mp.solutions.drawing_utils
mp_pose = mp.solutions.pose

In [12]:
def calculate_angle(a,b,c):
    a = np.array(a) # First
    b = np.array(b) # Mid
    c = np.array(c) # End
    
    radians = np.arctan2(c[1]-b[1], c[0]-b[0]) - np.arctan2(a[1]-b[1], a[0]-b[0])
    angle = np.abs(radians*180.0/np.pi)
    
    if angle >180.0:
        angle = 360-angle
        
    return angle 

In [13]:
def playsound(stage):
    tts = gTTS(text=stage, lang=language, slow=False)
    filename = 'temp.mp3'
    tts.save(filename)
    music = pyglet.media.load(filename, streaming=False)
    music.play()
    
#     sleep(music.duration)
    os.remove(filename)

In [19]:
cap = cv2.VideoCapture('Videos/squat2.mp4')

# Curl counter variables
stage = None
state = None
flag = None
time = datetime.datetime.now()
## Setup mediapipe instance
with mp_pose.Pose(min_detection_confidence=0.5, min_tracking_confidence=0.5) as pose:
    while cap.isOpened():
        ret, frame = cap.read()
        
        # Recolor image to RGB
        image = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        image.flags.writeable = False
      
        # Make detection
        results = pose.process(image)
    
        # Recolor back to BGR
        image.flags.writeable = True
        image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)
        
        # Extract landmarks
        try:
            landmarks = results.pose_landmarks.landmark
            
            # Get coordinates
            shoulder = [landmarks[mp_pose.PoseLandmark.LEFT_SHOULDER.value].x,landmarks[mp_pose.PoseLandmark.LEFT_SHOULDER.value].y]
            hip = [landmarks[mp_pose.PoseLandmark.LEFT_HIP.value].x,landmarks[mp_pose.PoseLandmark.LEFT_HIP.value].y]
            knee = [landmarks[mp_pose.PoseLandmark.LEFT_KNEE.value].x,landmarks[mp_pose.PoseLandmark.LEFT_KNEE.value].y]
            ankle = [landmarks[mp_pose.PoseLandmark.LEFT_ANKLE.value].x,landmarks[mp_pose.PoseLandmark.LEFT_ANKLE.value].y]
            leftfootidx = [landmarks[mp_pose.PoseLandmark.LEFT_FOOT_INDEX.value].x,landmarks[mp_pose.PoseLandmark.LEFT_FOOT_INDEX.value].y]
            
            
            # Calculate angle
            angle = calculate_angle(shoulder, hip, knee)
            angle2 = calculate_angle(hip, knee, ankle)
            # Visualize angle
            x,y,c = frame.shape
            cv2.putText(image, str(angle), 
                           tuple(np.multiply(hip, [y, x]).astype(int)), 
                           cv2.FONT_HERSHEY_SIMPLEX, 0.7, (30, 30, 30), 2, cv2.LINE_AA
                                )
            cv2.putText(image, str(angle2), 
                           tuple(np.multiply(knee, [y, x]).astype(int)), 
                           cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 30, 30), 2, cv2.LINE_AA
                                )
            if ((angle > 70) & (angle < 95)) & ((angle2 > 70) & (angle2 < 95)):
                stage = 'Good Squat'
                state = 2
                if abs(leftfootidx[0] - knee[0]) < 0.05:
                    stage = 'Good squat'
                else:
                    stage = 'Knees not in line with toes'
            elif (angle < 70) | (angle2 < 70):
                stage = 'Squatting more than required'
                state = 3
            elif (angle > 95) & (angle < 120) | (angle2 > 95) & (angle2 < 120):
                stage = 'Please squat a little bit more'
                state = 1
            else:
                stage = 'Not squatted'
                state = 0
                
                       
        except:
            pass
        
        # Render curl counter
        # Setup status box
       
        

        # Stage data
        if state == 2:
            cv2.rectangle(image, (0,0), (500,73), (245,117,16), -1)
            cv2.putText(image, 'STAGE', (65,12), 
                        cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0,0,0), 1, cv2.LINE_AA)
            cv2.putText(image, stage, 
                        (60,60), 
                        cv2.FONT_HERSHEY_SIMPLEX, 1.5, (255,255,255), 2, cv2.LINE_AA)
                
        
        # Render detections
        if state == 0:
            mp_drawing.draw_landmarks(image, results.pose_landmarks, mp_pose.POSE_CONNECTIONS,
                                    mp_drawing.DrawingSpec(color=(245,117,66), thickness=2, circle_radius=2), 
                                    mp_drawing.DrawingSpec(color=(245,66,230), thickness=4, circle_radius=2) 
                                     ) 
        elif state == 1:
            mp_drawing.draw_landmarks(image, results.pose_landmarks, mp_pose.POSE_CONNECTIONS,
                                    mp_drawing.DrawingSpec(color=(245,117,66), thickness=2, circle_radius=2), 
                                    mp_drawing.DrawingSpec(color=(45,235,242), thickness=4, circle_radius=2) 
                                     ) 
        elif state == 2:
            mp_drawing.draw_landmarks(image, results.pose_landmarks, mp_pose.POSE_CONNECTIONS,
                                    mp_drawing.DrawingSpec(color=(245,117,66), thickness=2, circle_radius=2), 
                                    mp_drawing.DrawingSpec(color=(43,255,128), thickness=4, circle_radius=2) 
                                     ) 
        else:
            mp_drawing.draw_landmarks(image, results.pose_landmarks, mp_pose.POSE_CONNECTIONS,
                                    mp_drawing.DrawingSpec(color=(245,117,66), thickness=2, circle_radius=2), 
                                    mp_drawing.DrawingSpec(color=(45,66,255), thickness=4, circle_radius=2) 
                                     ) 
        cv2.imshow('Mediapipe Feed', image)
        
        if cv2.waitKey(10) & 0xFF == ord('q'):
            break
        if (state == 2) & (flag!=2) & (datetime.datetime.now() - time > timedelta(seconds = 3)):
            time = datetime.datetime.now()
            flag = 2
            playsound(stage)
        elif (state == 3) & (flag != 3) & (flag==2) & (datetime.datetime.now() - time > timedelta(seconds = 3)):
            time = datetime.datetime.now()
            flag = 3
            playsound(stage)
        elif (state == 1) & (flag != 1) & (flag == 2) & (datetime.datetime.now() - time > timedelta(seconds = 3)):
            time = datetime.datetime.now()
            flag = 1
            playsound(stage)
        else:
            flag = 0
    cap.release()
    cv2.destroyAllWindows()

In [15]:
frame.shape

(480, 854, 3)

In [16]:
for lndmrk in mp_pose.PoseLandmark:
    print(lndmrk)

PoseLandmark.NOSE
PoseLandmark.LEFT_EYE_INNER
PoseLandmark.LEFT_EYE
PoseLandmark.LEFT_EYE_OUTER
PoseLandmark.RIGHT_EYE_INNER
PoseLandmark.RIGHT_EYE
PoseLandmark.RIGHT_EYE_OUTER
PoseLandmark.LEFT_EAR
PoseLandmark.RIGHT_EAR
PoseLandmark.MOUTH_LEFT
PoseLandmark.MOUTH_RIGHT
PoseLandmark.LEFT_SHOULDER
PoseLandmark.RIGHT_SHOULDER
PoseLandmark.LEFT_ELBOW
PoseLandmark.RIGHT_ELBOW
PoseLandmark.LEFT_WRIST
PoseLandmark.RIGHT_WRIST
PoseLandmark.LEFT_PINKY
PoseLandmark.RIGHT_PINKY
PoseLandmark.LEFT_INDEX
PoseLandmark.RIGHT_INDEX
PoseLandmark.LEFT_THUMB
PoseLandmark.RIGHT_THUMB
PoseLandmark.LEFT_HIP
PoseLandmark.RIGHT_HIP
PoseLandmark.LEFT_KNEE
PoseLandmark.RIGHT_KNEE
PoseLandmark.LEFT_ANKLE
PoseLandmark.RIGHT_ANKLE
PoseLandmark.LEFT_HEEL
PoseLandmark.RIGHT_HEEL
PoseLandmark.LEFT_FOOT_INDEX
PoseLandmark.RIGHT_FOOT_INDEX


In [10]:
!pip freeze

absl-py==1.0.0
argon2-cffi==21.3.0
argon2-cffi-bindings==21.2.0
asttokens==2.0.5
astunparse==1.6.3
attrs==21.4.0
backcall==0.2.0
black==21.12b0
bleach==4.1.0
cachetools==4.2.4
certifi==2021.10.8
cffi==1.15.0
charset-normalizer==2.0.10
click==8.0.3
colorama==0.4.4
cycler==0.11.0
debugpy==1.5.1
decorator==5.1.1
defusedxml==0.7.1
entrypoints==0.3
executing==0.8.2
flatbuffers==2.0
fonttools==4.28.5
gast==0.4.0
google-auth==2.3.3
google-auth-oauthlib==0.4.6
google-pasta==0.2.0
grpcio==1.43.0
gTTS==2.2.4
h5py==3.6.0
idna==3.3
importlib-metadata==4.10.1
ipykernel==6.7.0
ipython==8.0.1
ipython-genutils==0.2.0
ipywidgets==7.6.5
jedi==0.18.1
Jinja2==3.0.3
jsonschema==4.4.0
jupyter==1.0.0
jupyter-client==7.1.1
jupyter-console==6.4.0
jupyter-core==4.9.1
jupyterlab-pygments==0.1.2
jupyterlab-widgets==1.0.2
keras==2.7.0
Keras-Preprocessing==1.1.2
kiwisolver==1.3.2
libclang==12.0.0
Markdown==3.3.6
MarkupSafe==2.0.1
matplotlib==3.5.1
matplotlib-inline==0.1.3
mediapipe==0.8.10.1
mistune==0.8.4
mypy-ext